# Explore here

In [ ]:
import numpy as np, random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import os.path
import joblib
import sklearn
import sklearn.metrics as metrics
random.seed(42),

EDA
PASO1) OBTENCION Y LIMPIEZA DE DATOS:

In [ ]:
df_rd= pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv')
df_rd.to_csv("../data/raw/df_rd.csv", index=False)
df_rd.info()

In [ ]:
#1.1) FILTRADO DE NANS Y NULLS.
df_rd = df_rd.drop_duplicates().reset_index(drop = True)
df_rd = df_rd.fillna(0)
df_rd.head(4)

In [ ]:
#1.2) IDENTIFICACION TARGET Y ETIQUETACION DE PREDICTORAS.
#NUESTRO TARGET ES OUTCOME, VARIABLE QUE NOS INDICA CON 0 QUE NO TIENE DIABETES, Y UNO QUE SI TIENE.
#EL OBJETIVO ES REALIZAR UN MODELO QUE PREDIGA DADO UNOS DATOS SI EL PACIENTE TIENE DIABETES.
def vartype(df):
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    df_tipos = pd.DataFrame({'Tipo': ['Numérica']*len(num_cols) + ['Categórica']*len(cat_cols),
                            'Columna': num_cols + cat_cols})
    total_num = len(num_cols)
    total_cat = len(cat_cols)
    total_general = total_num + total_cat
    # Crear una lista de diccionarios para las nuevas filas
    new_rows = [{'Tipo': 'Total Numérico', 'Columna': total_num},
                {'Tipo': 'Total Categórico', 'Columna': total_cat},
                {'Tipo': 'Total General', 'Columna': total_general},]
    df_tipos = pd.concat([df_tipos, pd.DataFrame(new_rows)], ignore_index=True)
    return df_tipos
df_rd_vars= vartype(df_rd)
print(df_rd_vars)

In [ ]:
#1.3) SIMPLIFICACION DEL DATAFRAME.
#NOS SERVIREMOS DE LA FUNCIONALIDAD SELECTKBEST PARA SIMPLIFICAR EL DATAFRAME, SE SIRVE DE CRITERIOS MATEMATICOS PARA ESCOGER LAS PREDICTORAS.
#AL NO HABER CATEGORICAS NO USAREMOS EL MODO CHI2, SOLO EL F_reg.
#DE ENTRADA INTENTAREMOS REDUCIR EL NUMERO DE PREDICTORAS DE 8 A 6.
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
# Suponiendo que 'df_rd' es tu DataFrame y 'target' es la columna objetivo
X = df_rd.drop('Outcome', axis=1)  # Características
y = df_rd['Outcome']  # Variable objetivo
# Seleccionar las 6 mejores características (ajusta el valor de k según tus necesidades)
selector = SelectKBest(f_regression, k=6)
X_new = selector.fit_transform(X, y)
# Obtener las características seleccionadas
selected_features = X.columns[selector.get_support()]
# Crear un nuevo DataFrame con las características seleccionadas y la variable objetivo
df_Eda = pd.DataFrame(X_new, columns=selected_features)
df_Edav7 = pd.concat([df_Eda, y], axis=1)  #Agregamos manualmente la target y al dataframe para completarlo.
df_Edav7.to_csv('../data/processed/df_edav7.csv', index=False)
# Visualizar las primeras filas del DataFrame
print(df_Edav7.describe())



Nuestro dataframe resultante es tal y como queriamos, 6 predictoras y nuestra 'Outcome' como var target, total 7 vars.(df_Edav7, df_Eda(es el dataframe procesado de df_rd(dataframe_RawData)) con 7 variables(v7))
Podemos echar un vistazo al heatmap para ver correlaciones, y usando criterio de correlacion simplificarlo aun mas.(No hay necesidad de hacerlo en este caso, ya hay pocas variables.)

1.4)FILTRADO DF: Bifurcacion dataframe con outliers(col), y sin outliers(sol), guardando datos atipicos en diccionarios por variable(fmt Json)

In [ ]:
#dataframe sin outliers.
df_E7_sol= df_Edav7.copy()
df_E7_sol.to_csv('../data/processed/df_E7_sol.csv', index=False)
#dataframe con outliers.
df_E7_col= df_Edav7.copy()
df_E7_col.to_csv('../data/processed/df_E7_col.csv', index=False)

#3.2) guardar outliers en la carpeta lims
def LimsOlSaveJson(df, columns, output_path, lower_limit_zero=True):
    for column in columns:
        stats = df[column].describe()
        iqr = stats['75%'] - stats['25%']
        ul = stats['75%'] + (2 * iqr)
        ll = max(stats['25%'] - (2 * iqr), 0 if lower_limit_zero else None)
        limits = {"upper_limit": ul, "lower_limit": ll}

        # Traducir signos especiales que dan error a la hora de guardar
        safe_column_name = column.replace(' ', '_').replace('/', '_').replace('%', '_')

        file_path = f"{output_path}/limits_{safe_column_name}.json"
        with open(file_path, "w") as f:
            json.dump(limits, f, indent=4)

        df[column] = df[column].apply(lambda x: ul if x > ul else ll if x < ll else x)

    return df


num_vars = df_E7_sol.select_dtypes(include=np.number).columns.tolist()
LimsOlSaveJson(df_E7_sol, num_vars, '../data/processed/lims')
df_E7_sol.describe()

Trabajaremos con el dataframe sin outliers, alguno pensara que siendo personas no se deberia hacer, pero hay valores biologicos surrealistas como valores atipicos,
mejor entrenar un modelo que se ajuste a la gran mayoria de personas.
Tamnbien he decidio trabajar con datos originales sin Normalizar/estandarizar.

In [ ]:
#PASO 2) ANÁLISIS UNIVARIABLE NUMERICA.
import matplotlib.pyplot as plt
import seaborn as sns
import math
def GrafUniNum(df_no_outliers, df_with_outliers, hist_vars, box_vars):
  """
  Creamos un conjunto de graficos, los histogramas sin outliers para presenciar mejor la distribucion,
  los graficos de caja con outliers para visualizarlos.
  Args:
      df_no_outliers
      df_with_outliers
      hist_vars: lista de nombres de columnas.
      box_vars: lista de nombres de columnas.
  """

  num_hist_vars = len(hist_vars)  #Determinamos el numero de variables para el histograma sin Ol's
  num_box_vars = len(box_vars)  #Determinamos el numero de variables para los graficos de caja con Ol's
  total_vars = num_hist_vars + num_box_vars

  #Determinamos la cuadricula de graficos optima, evitando el zero
  if total_vars <= 4:
    row, col = 2, 2  
  elif total_vars <= 6:
    row, col = 2, 3  
  else:
    # Usaremos una rejilla de graficos variable en caso de exceder los casos anteriores.
    row = math.ceil(total_vars / 3)
    col = 3
  fig, axs = plt.subplots(row, col, figsize=(20, 5 * row))
  i = 0
  for var in hist_vars:
    row_index = i // col
    col_index = i % col
    sns.histplot(ax=axs[row_index, col_index], data=df_no_outliers, x=var, color="blue")
    max_val_no_outliers = df_no_outliers[var].max()
    axs[row_index, col_index].set_xlim(0, max_val_no_outliers)
    axs[row_index, col_index].set_xticklabels(axs[row_index, col_index].get_xticklabels(), rotation=90)
    i += 1
  for var in box_vars:
    row_index = i // col
    col_index = i % col
    sns.boxplot(ax=axs[row_index, col_index], data=df_with_outliers, x=var, color="red")
    axs[row_index, col_index].set_xticklabels(axs[row_index, col_index].get_xticklabels(), rotation=90)
    i += 1
  plt.tight_layout()
  plt.show()
#Preparamos la llamada a la funcion y el input
hist_vars = df_E7_sol.columns
box_vars = df_E7_col.columns
GrafUniNum(df_E7_sol, df_E7_col, hist_vars, box_vars)

Vemos varias distribuciones predominantes, como es obvio en variables como pregnancies o insulina, donde normalmente se tienen pocos hijos o se tienen valores  relativamente bajos de insulina cuando estas sano, se observa una distribucion normal asimetrica,
desplazada a valores cerca del 0. Es decir el mayor porcentaje de muestras se ubican cuando el rango de estas dos variables se aproxima a 0.
El mismo caso pasa con Age, pero empieza esa distribucion cuando el valor del rango age vale 20, entendemos que es porque hasta que las muestras no tienen esa edad, no figuran registrados.
BMi y pedigree diabetes function tienen una distribucion casi simetrica en el centro del rango de valor de estas dos variables.
En cuanto a los outliers, es un data set bastante limpio donde hay pocas muestras fuera de los rangos cuartilicos y alejados del maximo.

In [ ]:
#PASO 4) ANÁLISIS MULTIVARIABLE NUMERICO-NUMERICO.
print('Multivariable sin outliers df valores reales')
sns.pairplot(df_E7_sol)
plt.show()

In [ ]:
#PASO 4)MAPA DE CORRELACIÓN. MAPA DE CALOR.

target_var = "Outcome"  
df_heatmap = df_E7_sol.reindex(columns=[target_var] + list(df_E7_sol.columns.difference([target_var])))
#Ahora representamos el grafico.
plt.figure(figsize=(18, 12))
sns.heatmap(df_heatmap.corr(), annot=True, fmt=".2f")
plt.show()

COMO PODEMOS OBSERVAR EL SELECTKBEST PARA REALIZAR EL EDA AJUSTADO FUNCIONA DE MARAVILLA. PODRIAMOS HACER ALGO CON LAS COLUMNAS PREGNANCIES Y AGE PERO YA TENEMOS UN NUMERO BAJO DE VARIABLES.
Poco mas que añadir, observamos correlaciones leves y alguna bastante significativa como Age y pregnancies(0.55).
Con respecto a la variable TARGET hay algunas con coeficientes superiores al 0.20 y solo dos con correlacion negativa, muy leves por cierto.
Creo que con estos datos se podra hacer un modelo de arbol de decision bastante bueno.

PASO 5) FEATURE ENGINEERING.

In [105]:
#PASO 5) FUNCION SPLIT TRAIN/TEST:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

def split_and_save_data(df, target_column, output_dir, test_size=0.20, random_state=42, filename_prefix=""):
  X = df.drop(columns=target_column)
  y = df[target_column]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
  train_dir = os.path.join(output_dir, "train")
  test_dir = os.path.join(output_dir, "test")
  os.makedirs(train_dir, exist_ok=True)
  os.makedirs(test_dir, exist_ok=True)
  X_train.to_csv(os.path.join(train_dir, f"{filename_prefix}_X_train.csv"), index=False)
  X_test.to_csv(os.path.join(test_dir, f"{filename_prefix}_X_test.csv"), index=False)
  y_train.to_csv(os.path.join(train_dir, f"{filename_prefix}_y_train.csv"), index=False)
  y_test.to_csv(os.path.join(test_dir, f"{filename_prefix}_y_test.csv"), index=False)
output_dir = '../data/processed/Split'
target_column = 'Outcome'
split_and_save_data(df_E7_sol, target_column, output_dir, filename_prefix="rawsol")


In [106]:
#PASO 5) FEATURE ENGINEERING, DECISION TREE:
#PREPARACION DE DATOS
#Dataframes para representacion de graficos(Incluyendo la var target de nuevo como columna del df con X train/test).
X_train = pd.read_csv('../data/processed/Split/train/rawsol_X_train.csv', index_col=False).copy()
y_train = pd.read_csv('../data/processed/Split/train/rawsol_y_train.csv', index_col=False).copy()
X_test = pd.read_csv('../data/processed/Split/test/rawsol_X_test.csv', index_col=False).copy()
y_test = pd.read_csv('../data/processed/Split/test/rawsol_y_test.csv', index_col=False).copy()
#DF's
df_X_train = pd.DataFrame(X_train.copy(), columns=['Pregnancies','Glucose', 'Insulin',	'BMI', 'DiabetesPedigreeFunction','Age'])
df_X_test = pd.DataFrame(X_test.copy(), columns=['Pregnancies','Glucose', 'Insulin',	'BMI', 'DiabetesPedigreeFunction','Age'])
df_X_train['Outcome'] = y_train.copy()
df_X_test["Outcome"]= y_test.copy()

#Arrays para el calculo sin formatos, indices o indices de columnas
a_X_train = pd.read_csv('../data/processed/Split/train/rawsol_X_train.csv', index_col=False).copy().values
a_y_train = pd.read_csv('../data/processed/Split/train/rawsol_y_train.csv', index_col=False).copy().values
a_X_test = pd.read_csv('../data/processed/Split/test/rawsol_X_test.csv', index_col=False).copy().values
a_y_test = pd.read_csv('../data/processed/Split/test/rawsol_y_test.csv', index_col=False).copy().values


In [ ]:
#Paso 5) Representacion grafico lineas paralelas.
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.figure(figsize=(12, 6))
colors = ("#fd2e02","#8a493b","#fa9904","#9d7639","#ffd800","#ac9b3b","#b0fb03","#7e9942","#57fe04","#5d9b3e","#03fb25","#399c47","#03fee3","#349c91","#029dfb","#3f7ea4","#3f7ea4","#395d9b","#4b05f9","#543d8e","#8700ff","#683892","#f300ff","#863f89","#fb048b","#893f68")
linewidth = 1
pd.plotting.parallel_coordinates(df_E7_col, "Outcome", color=colors, linewidth=linewidth)
plt.show()

In [ ]:
#PASO 5) MODELO ARBOL DE DECISION.
#Este modelo trabaja categorizando la informacion, ademas de que has de presentar los segmentos de TRAIN/TEST en formato dataframe,
#Hay que tener cuidado a la hora de introducir los inputs.
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state = 42)
model.fit(df_X_train.copy().drop(['Outcome'],axis=1), df_X_train['Outcome'].copy())
y_test = df_X_test['Outcome'].copy()
#Vamos a diseñar un modelo de arbol sencillo para nuestra variable target, en su parametrizacion, class_name(etiquetado categorico y resolucion de la prediccion de una hoja o decision de pasar al siguiente nodo)
#Si el arbol concluye class name = 0, querra decir que la muestra predecida no tiene diabetes y por lo tanto el arbol termina en esa Hoja "leaf"
#si el arbol concluye class name = 1, querra decir que la muestra predecida si tiene diabetes y por lo tanto el arbol termina en esa Hoja "leaf"
#si el arbol concluye class name = 2, querra decir que la muestra predecida aun se desconoce su clase en su variable predict y "Outcome" por lo tanto el arbol ir'a al siguiente Nodo y bajara un nivel.
from sklearn import tree
fig = plt.figure(figsize=(15,15))
tree.plot_tree(model, feature_names = list(df_X_train.columns.copy()), class_names = ["0", "1", "2"], filled = True)
plt.show()



Realizamos las observaciones necesarias para sacar un acuracy score respecto a los datos entrenados y los predecidos:

In [ ]:
#PASO 5) ACURACY SCORE.
y_pred = model.predict(df_X_test.drop(['Outcome'].copy(), axis=1))
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(a_y_test, y_pred)

PASO 5.1) OPTIMIZACION MODELO , AJUSTE HIPERPARAMETROS via GRIDSEARCH, RANDOMSEARCH y Libreria HiperOPT.
A partir de este momento, toda la mejora se haran con copias del modelo base.


In [111]:
#5.1.1)OPTIMIZACION HIPERPARAMETROS VIA GRIDSEARCH
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

hyperparams = {
    "criterion": ["gini", "entropy"], #Criterio de División: Determina la calidad de una división. 
                                      #- gini: Impureza de Gini mide la probabilidad de clasificar 
                                      #erróneamente un punto de datos si fuera elegido al azar del nodo actual.
                                      #- entropy: Ganancia de información mide la reducción 
                                      # en la incertidumbre después de una división.
                                      
    "max_depth": [None, 6, 36, 216],   #Profundidad Máxima: Controla la profundidad máxima del árbol.
                                      #Un árbol más profundo puede capturar patrones más complejos pero es propenso al sobreajuste.
                                     
    "min_samples_split": [3, 9, 27],  #Muestras Mínimas para Dividir: Especifica el número mínimo de muestras 
                                      #requeridas para dividir un nodo interno. Un valor más alto puede prevenir el sobreajuste.
                                    
    "min_samples_leaf": [3, 6, 9]     #Muestras Mínimas por Hoja: Establece el número mínimo de muestras requeridas 
                                      #para estar en una hoja. Un valor más alto puede prevenir el sobreajuste y hacer que el árbol sea más robusto.
}



In [ ]:
grid = GridSearchCV(model, hyperparams, scoring = "accuracy", cv = 9)
grid 

#Parámetros clave en GridSearchCV:
#MODEL:
#Seleccionamos el modelo que queremos mejorar
#En este caso, un DecisionTreeClassifier.
#PARAM_GRID(hyperparams):
#Este diccionario especifica los diferentes valores que ajustamos para cada hiperparámetro.
#Probamos diferentes combinaciones de criterion, max_depth, min_samples_split, y min_samples_leaf.
#SCORING:
#Este parámetro define la métrica de evaluación que se utiliza para comparar diferentes combinaciones de hiperparámetros.
#En este caso, se utiliza la "accuracy" (precisión) para evaluar el rendimiento del modelo.
#CV:
#Este parámetro especifica el número de pliegues de validación cruzada que utilizamos para evaluar el modelo.
#En este caso, se utiliza una validación cruzada de 10 pliegues, lo que significa que los datos se dividen en 10 partes,
#se entrenan modelos en 9 partes y se evalúan en la parte restante, y esto se repite 10 veces.


In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
grid.fit(X_train.copy(), y_train.copy())
print(f"Mejor combo de hiperparametros: {grid.best_params_}")

In [ ]:
#Ahora ajustamos nuestro modelo decision tree con los parametros ajustados obtenidos por gridsearch
model = DecisionTreeClassifier(criterion = "gini", max_depth = None, min_samples_leaf = 9, min_samples_split = 27, random_state = 42)
model.fit(X_train.copy(), y_train.copy())

In [ ]:
#Realizamos el test
y_pred2= model.predict(X_test.copy())
y_pred2

In [ ]:
from sklearn.metrics import accuracy_score
from pickle import dump
accuracy = accuracy_score(y_test.copy(), y_pred2)
print("Accuracy:", accuracy)


In [117]:
#GUARDAMOS RESULTADOS GRIDSEARCH.
dump(model, open("../models/3c_G_MDn_MSL9_MSS27.sav", "wb")) # model = DecisionTreeClassifier(criterion = "gini", max_depth = None, min_samples_leaf = 9, min_samples_split = 27, random_state = 42)

HEMOS TENIDO UNA GANANCIA POCO SIGNIFICATIVA ALREDEDOR DE UN +0.014

In [ ]:
#5.1.2)OPTIMIZACION HIPERPARAMETROS VIA RANDOMSEARCH, IMPORTANTE REALIZAR MEJORA CON COPIAS DE LAS VARIABLES SPLIT TRAIN/TEST PARA NO SOBREESCRIBIR Y NO ACUMULAR SOBREENTRENO
#Solo se puede mejorar una vez desde que hicimos el decision tree default. es lo optimo.
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint


# Espacio de búsqueda de hiperparámetros
param_dist = {
    'max_depth': randint(0,6),
    'max_features': randint(1, 9),
    'min_samples_leaf': randint(1, 9),
    'criterion': ['gini', 'entropy']
}

# Crear el objeto RandomizedSearchCV
random_search = RandomizedSearchCV(model, param_distributions=param_dist, cv=6, n_iter=9)
# Ajustar el modelo
random_search.fit(X_train.copy(), y_train.copy())
# Obtener los mejores hiperparámetros
best_params = random_search.best_params_
print(best_params)


In [ ]:
#Ahora ajustamos nuestro modelo decision tree con los parametros ajustados obtenidos por randomsearch
model = DecisionTreeClassifier(criterion = "entropy", max_depth = 5, max_features = 8, min_samples_leaf = 6,random_state = 42)
model.fit(X_train.copy(), y_train.copy())

In [ ]:
#Prediccion via RandomSearch
y_pred3 = model.predict(X_test.copy())
y_pred3

In [ ]:
from sklearn.metrics import accuracy_score
from pickle import dump
accuracy = accuracy_score(y_test.copy(), y_pred3)
print("Accuracy:", accuracy)

In [122]:
#GUARDAMOS RESULTADOS. RANDOMSEARCH
dump(model, open("../models/3c_E_MD5_MF8_MSF6.sav", "wb")) # model = DecisionTreeClassifier(criterion = "entropy", max_depth = 5, max_features = 8, min_samples_leaf = 6,random_state = 42)

HEMOS MEJORADO SIGNIFICATIVAMENTE NUESTRO MODELO SIN SOBREENTRENO, DADO QUE HEMOS USADO LOS DATOS BASE TRAS EL PRIMER MODELADO.
DE 0.7142857142857143 EN DEFAULT A 0.7727272727272727 CON RANDOMSEARCH.

In [ ]:
#5.1.2)OPTIMIZACION HIPERPARAMETROS VIA LIBRERIA HIPEROPT
from hyperopt import fmin, tpe, hp
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

def objective(space):
    clf = DecisionTreeClassifier(
        max_depth=space['max_depth'],
        min_samples_split=int(space['min_samples_split']),  # valores enteros solo sin decs.
        min_samples_leaf=int(space['min_samples_leaf']),  
        criterion=space['criterion'],
        max_features=int(space['max_features']),
        #class_weight=space['class_weight'],
        #ccp_alpha=space['ccp_alpha']
    )
    scores = cross_val_score(clf, X, y, cv=5) #EL cv lo podemos modificar tambien.
    return -scores.mean()  # Maximize accuracy

space = { #hay que indicar explicitamente instrucciones que hagan referencia a obtencion de valores enteros sin decimales.
          #ya que los parametros del decision tree son enteros.
    'max_depth': hp.choice('max_depth', range(1, 21)), #Indicamos el rango de max depth
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1), #Indicamos el rango de msp
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5, 1), #Indicamos el rango de msf
    'criterion': hp.choice('criterion', ['gini', 'entropy']), #Indicamos el rango categorico de criterion
    'max_features': hp.quniform('max_features', 1, 9, 1), #Indicamos el rango de mf
    #'class_weight': hp.choice('class_weight', [{0: 1, 1: 1}, {0: 1, 1: 2}, {0: 1, 1: 3}]), #esta la obviamos porque nuestra clase ya esta definida en el punto de partida
    #'ccp_alpha': hp.uniform('ccp_alpha', 0.0, 0.1) #no queremos modificar el alpha, hace referencia al ajuste lineal de las variables, podemos modificarlo, pero habria que mirar que hace exactamente.
}

best = fmin(objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100)

print(best)

In [ ]:
#Ahora ajustamos nuestro modelo decision tree con los parametros ajustados obtenidos por la libreria HyperOpt.
model = DecisionTreeClassifier(criterion = "gini", max_depth = 45, max_features = 8, min_samples_leaf = 4, min_samples_split=7, random_state = 42)
model.fit(X_train.copy(), y_train.copy())

In [ ]:
#Realizamos la prediccion.
y_pred4 = model.predict(X_test.copy())
y_pred4

In [ ]:
from sklearn.metrics import accuracy_score
from pickle import dump
accuracy = accuracy_score(y_test.copy(), y_pred4)
print("Accuracy:", accuracy)


In [127]:
#GUARDAMOS RESULTADOS. hyperopt
dump(model, open("../models/3c_G_MD4_MF8_MSL4_MSS7.sav", "wb")) #{'criterion': np.int64(0), 'max_depth': np.int64(4), 'max_features': np.float64(8.0), 'min_samples_leaf': np.float64(4.0), 'min_samples_split': np.float64(7.0)}

5.2) FEATURE ENGINEERING: RANDOM FOREST

Podemos construir un árbol de decisión fácilmente en Python utilizando la librería scikit-learn y las funciones RandomForestClassifier y RandomForestRegressor. 
Algunos de sus hiperparámetros más importantes y los primeros en los que debemos centrarnos son:
n_estimators: Este es probablemente el hiperparámetro más importante. Define el número de árboles de decisión en el bosque. En general, un número mayor de árboles aumenta la precisión y hace que las predicciones sean más estables, pero también puede ralentizar considerablemente el tiempo de cálculo.

bootstrap: Este hiperparámetro se usa para controlar si se utilizan muestras de bootstrap (muestreo con reemplazo) para la construcción de árboles.

max_depth: La profundidad máxima de los árboles. Esto es esencialmente cuántas divisiones puede hacer el árbol antes de hacer una predicción.

min_samples_split: El número mínimo de muestras necesarias para dividir un nodo en cada árbol. Si se establece un valor alto, evita que el modelo aprenda relaciones demasiado específicas y, por tanto, ayuda a prevenir el sobreajuste.

min_samples_leaf: El número mínimo de muestras que se deben tener en un nodo hoja en cada árbol.

max_features: El número máximo de características a considerar al buscar la mejor división dentro de cada árbol. Por ejemplo, si tenemos 10 características, podemos elegir que cada árbol considere solo un subconjunto de ellas al decidir dónde dividir.

Como podemos ver, solo los dos primeros hiperparámetros hacen referencia al random forest, mientras que el resto se relacionan con los árboles de decisión. Otro hiperparámetro muy importante es el random_state, que controla la semilla de generación aleatoria. Este atributo es crucial para asegurar la replicabilidad.

In [ ]:
#5.2) RANDOM FOREST
X_trRF = pd.read_csv('../data/processed/Split/train/rawsol_X_train.csv', index_col=False).copy()
y_trRF = pd.read_csv('../data/processed/Split/train/rawsol_y_train.csv', index_col=False).copy()
X_tRF = pd.read_csv('../data/processed/Split/test/rawsol_X_test.csv', index_col=False).copy()
y_tRF = pd.read_csv('../data/processed/Split/test/rawsol_y_test.csv', index_col=False).copy()
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(
    n_estimators=100,  # Número de árboles en el bosque
    max_depth=6,       # Profundidad máxima de cada árbol
    random_state=42, #Semilla de orden aleatorio.
    min_samples_split=12, #El número mínimo de muestras necesarias para dividir un nodo
    min_samples_leaf=4, #El número mínimo de muestras que se deben tener en un nodo hoja en cada árbol
    max_features=5, #El número máximo de características a considerar al buscar la mejor división dentro de cada árbol
)
model = RandomForestClassifier(random_state = 42)
model.fit(X_trRF, y_trRF)

In [ ]:
fig, axis = plt.subplots(2, 2, figsize = (20, 15))
# Mostramos los 4 primeros árboles de los 100 gen.
tree.plot_tree(model.estimators_[0], ax = axis[0, 0], feature_names = list(X_trRF.copy().columns), class_names = ["0", "1", "2"], filled = True)
tree.plot_tree(model.estimators_[1], ax = axis[0, 1], feature_names = list(X_trRF.copy().columns), class_names = ["0", "1", "2"], filled = True)
tree.plot_tree(model.estimators_[2], ax = axis[1, 0], feature_names = list(X_trRF.copy().columns), class_names = ["0", "1", "2"], filled = True)
tree.plot_tree(model.estimators_[3], ax = axis[1, 1], feature_names = list(X_trRF.copy().columns), class_names = ["0", "1", "2"], filled = True)
plt.show()

In [ ]:
y_predRF = model.predict(X_tRF)
y_predRF

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_tRF, y_predRF)